In [1]:
import os
import shutil
import subprocess
import traceback
import argparse
from time import time
import datetime
import pandas as pd
import numpy as np
import cv2, re
import matplotlib.pyplot as plt
from ast import literal_eval
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from sys import exit
from tqdm import tqdm
from natsort import natsorted
from pyglet.canvas import Display
from scipy import stats
from tkinter import filedialog as fd
from tkinter import Tk
from skimage import io
from skimage.morphology import skeletonize, remove_small_objects
from skimage.filters import (
      threshold_otsu, threshold_local, threshold_multiotsu, gaussian,
      apply_hysteresis_threshold,try_all_threshold, threshold_minimum,
      threshold_yen, threshold_li, threshold_isodata, threshold_triangle,
      try_all_threshold)
from skimage.color import label2rgb
from skimage.exposure import histogram, equalize_adapthist
from skimage.feature import peak_local_max
from skimage.measure import label, regionprops
from load import load_data
from prompts import select_channel_name
from apps import inspection_app, imshow_tk, inspect_effect_size, tk_breakpoint
import core

import matplotlib
# The script will create plots that will be saved to pdf. No need to use
# a GUI backend here.
matplotlib.use('Agg')

In [2]:
# Script version (to append for file saving)
vNUM = f'v1'
#Prompt the user to select the image file
# Folder dialog
selected_path = core.folder_dialog(title=
    'Select folder with multiple experiments, the TIFFs folder or '
    'a specific Position_n folder')

if not selected_path:
    exit('Execution aborted.')

selector = core.select_exp_folder()
is_pos_path = os.path.basename(selected_path).find('Position_') != -1
is_TIFFs_path = os.path.basename(selected_path).find('TIFFs') != -1
if not is_pos_path and not is_TIFFs_path:
    beyond_listdir = core.beyond_listdir_mitoQUANT(selected_path, vNUM=vNUM)
    selector.run_widget(
             beyond_listdir.all_exp_info,
             title='mitoQUANT: Select experiment to analyse',
             label_txt='Select experiment to analyse',
             full_paths=beyond_listdir.TIFFs_path,
             showinexplorer_button=True,
             all_button=True,
             remaining_button=True,
             selected_path=selected_path,
             NONnucleoCOUNTED_pos=beyond_listdir.NONnucleoCOUNTED_pos_info,
             NONnucleoSIZED_pos=beyond_listdir.NONnucleoSIZED_pos_info,
             NONnucleoGAUSSfit_pos=beyond_listdir.NONnucleoGAUSSfit_pos_info)
    main_paths = selector.paths
    prompts_pos_to_analyse = False
else:
    # The selected path is already the folder containing Position_n folders
    prompts_pos_to_analyse = True
    main_paths = [selected_path]

# Channel name. The load_data selector is used by load_data class to
# prompt to select a channel name from available channel names in the metadata
# if the channel name selected by the user is not present.
ch_name_selector = select_channel_name()
load_data_ch0_selector = select_channel_name()

ref_ch_name_selector = select_channel_name()
load_data_ref_ch_selector = select_channel_name()

In [9]:
inspect = True
verbose = 1
script_dirpath = r'G:\My Drive\01_Postdoc_HMGU\Python_MyScripts\MIA\Git\mitoQUANT\src'

In [ ]:
t0_tot = time()
"""Iterate experiment folders"""
for exp_idx, main_path in enumerate(main_paths):
    t0_exp = time()

    # The selected path is either a single path (endint with TIFFs) or
    # a list of Postion_n paths (ending in Position_n)
    if isinstance(main_path, str):
        dirname = os.path.basename(main_path)
        TIFFs_path = main_path
    elif isinstance(main_path, list):
        dirname = os.path.basename(main_path[0])
        TIFFs_path = os.path.dirname(main_path[0])


    if dirname == 'TIFFs':
        print('')
        print('##################################################')
        print(f'Analysing experiment: {os.path.dirname(main_path)}')
        folders_main_path = [f for f in natsorted(os.listdir(main_path))
                                    if os.path.isdir(f'{main_path}/{f}')]

        paths = []
        for i, d in enumerate(folders_main_path):
            images_path = f'{main_path}/{d}/Images'
            filenames = os.listdir(images_path)
            if ch_name_selector.is_first_call:
                ch_names = ch_name_selector.get_available_channels(filenames)
                ch_name_selector.prompt(ch_names)
                channel_name = ch_name_selector.channel_name
            spots_ch_aligned_found = False
            for j, f in enumerate(filenames):
                if f.find(f'_{channel_name}_aligned.npy') != -1:
                    spots_ch_aligned_found = True
                    aligned_i = j
                elif f.find(f'_{channel_name}.tif') != -1:
                    tif_i = j
            if spots_ch_aligned_found:
                spots_ch_path = os.path.join(images_path, filenames[aligned_i])
            else:
                spots_ch_path = os.path.join(images_path, filenames[tif_i])
            paths.append(spots_ch_path)

        ps, pe = 0, len(paths)
        if exp_idx == 0:
            mitoQ_inputs_widget = core.spotQUANT_inputs_widget()
            # buttons = core.fourbuttonsmessagebox()
            if prompts_pos_to_analyse:
                ps, pe = core.num_pos_toQuant_tk(len(paths)).frange

    elif is_pos_path:
        pos_path = main_path
        images_path = f'{pos_path}/Images'
        filenames = os.listdir(images_path)
        if ch_name_selector.is_first_call:
            ch_names = ch_name_selector.get_available_channels(filenames)
            ch_name_selector.prompt(ch_names)
            channel_name = ch_name_selector.channel_name
        spots_ch_aligned_found = False
        for j, f in enumerate(filenames):
            if f.find(f'_{channel_name}_aligned.npy') != -1:
                spots_ch_aligned_found = True
                aligned_i = j
            elif f.find(f'_{channel_name}.tif') != -1:
                tif_i = j
        if spots_ch_aligned_found:
            spots_ch_path = os.path.join(images_path, filenames[aligned_i])
        else:
            spots_ch_path = os.path.join(images_path, filenames[tif_i])
        paths = [spots_ch_path]
        ps, pe = 0, len(paths)
        if exp_idx == 0:
            # buttons = core.fourbuttonsmessagebox()
            mitoQ_inputs_widget = core.spotQUANT_inputs_widget()
    else:
        raise FileNotFoundError('The path selected is not a valid path!')

    user_ch_name = channel_name

    """Iterate Position folders"""
    num_pos_total = len(paths[ps:pe])
    for pos_idx, path in enumerate(paths[ps:pe]):
        t0_pos = time()
        pos_path = os.path.dirname(os.path.dirname(path))
        pos_foldername = os.path.basename(pos_path)
        print('')
        print('----------------------------------')
        print(f'Analysing experiment {exp_idx+1}/{len(main_paths)}, '
              f'{pos_foldername} ({pos_idx+1}/{num_pos_total})')
        print(f'{pos_path}...')
        print('')

        t0 = time()

        #Load tif file images and metadata
        print('Loading data...')
        spots_ch_data = load_data(path, channel_name, load_shifts=True,
                               load_segm_npy=True,
                               load_cca_df=True, create_data_folder=True,
                               ch_name_selector=load_data_ch0_selector)

        channel_name = spots_ch_data.channel_name
        cca_df = spots_ch_data.cca_df

        # Show mitoQUANT input variables to the user
        if mitoQ_inputs_widget.show:
            vox_dim = [round(r, 5) for r in  spots_ch_data.zyx_vox_dim]
            mitoQ_inputs_widget.run(
                      title='mitoQUANT analysis inputs',
                      channel_name=channel_name,
                      zyx_voxel_size=f'{vox_dim}',
                      zyx_voxel_size_float=spots_ch_data.zyx_vox_dim,
                      z_resolution_limit='1',
                      numerical_aperture=f'{spots_ch_data.NA}',
                      em_wavelength=f'{spots_ch_data.wavelen}',
                      gauss_sigma='0.75',
            )

            local_maxima_thresh_func = globals()[mitoQ_inputs_widget
                                                 .local_max_thresh_func]
            ref_ch_thresh_func = globals()[mitoQ_inputs_widget
                                          .ref_ch_thresh_func]
            zyx_vox_dim = literal_eval(mitoQ_inputs_widget.zyx_vox_size)
            z_resolution_limit = float(mitoQ_inputs_widget.z_resol_limit)
            NA = float(mitoQ_inputs_widget.NA)
            wavelen = float(mitoQ_inputs_widget.em_wavel)
            sigma = float(mitoQ_inputs_widget.gauss_sigma)
            yx_resolution_multiplier = float(mitoQ_inputs_widget
                                             .yx_resolution_multiplier)

            gop_how = mitoQ_inputs_widget.gop_how
            which_effsize = mitoQ_inputs_widget.which_effsize
            gop_thresh_val = [float(mitoQ_inputs_widget.gop_limit_txt)]
            do_bootstrap = gop_how.find('bootstrap') != -1

            calc_mtNet_len = mitoQ_inputs_widget.calc_mtNet_len
            do_nucleoSIZE = mitoQ_inputs_widget.do_nucleoSIZE
            do_gaussian_fit = mitoQ_inputs_widget.do_gaussian_fit

            load_ref_ch = mitoQ_inputs_widget.load_ref_ch
            filter_by_ref_ch = mitoQ_inputs_widget.filter_by_ref_ch

            do_save = mitoQ_inputs_widget.save

            df_inputs = pd.DataFrame({
                             'Description': mitoQ_inputs_widget.entry_labels,
                             'Values': mitoQ_inputs_widget.entry_txts}
                              ).set_index('Description').sort_index()

            # Store last status of the inputs widget to load it next time
            df_inputs.to_csv(os.path.join(script_dirpath,
                                          'last_status_inputs_widget.csv'))

            replace = False
            if os.path.exists(spots_ch_data.data_path) and do_save:
                if len(os.listdir(spots_ch_data.data_path)) > 5:
                    do_save, replace = (mitoQ_inputs_widget
                            .check_if_same_of_prev_run(data_path,
                                                       vNUM, df_inputs)
                )

            run_num = mitoQ_inputs_widget.run_num

        # Prompt to select reference channel name
        if load_ref_ch and ref_ch_name_selector.is_first_call:
            images_path = os.path.dirname(path)
            filenames = os.listdir(images_path)
            ch_names = ref_ch_name_selector.get_available_channels(filenames)
            ch_names = [c for c in ch_names if c.find(user_ch_name)==-1]
            ref_ch_name_selector.prompt(ch_names,
                                        message='Select reference channel name')
            ref_channel_name = ref_ch_name_selector.channel_name

        #Align frames according to saved shifts (calculated from the phase contrast)
        if not spots_ch_data.already_aligned:
            print('Aligning frames...')
            spots_ch_aligned = core.align_frames(spots_ch_data.frames,
                                         path=spots_ch_data.path,
                                         save_aligned=True, register=False,
                                         saved_shifts=spots_ch_data.shifts)
            if load_ref_ch:
                ref_ch_filename = f'{ref_channel_name}.tif'
        elif spots_ch_data.skip_alignment:
            if load_ref_ch:
                ref_ch_filename = f'{ref_channel_name}.tif'
        else:
            if load_ref_ch:
                ref_ch_filename = f'{ref_channel_name}_aligned.npy'

        if load_ref_ch:
            #Get path to mKate file
            position_n_path = os.path.dirname(path)
            found = False
            for filename in os.listdir(position_n_path):
                if filename.find(ref_ch_filename)>0:
                    found=True
                    break
            if not found:
                raise FileNotFoundError(ref_ch_filename+' Not found.')
            ref_ch_path = os.path.join(position_n_path, filename)

            #Load reference channel data and metadata
            ref_ch_data = load_data(ref_ch_path, ref_channel_name,
                               load_metadata=not spots_ch_data.metadata_found,
                               load_mask=False,
                               ch_name_selector=load_data_ref_ch_selector)

            #Align frames according to saved shifts (calculated from the phase contrast)
            if not spots_ch_data.already_aligned:
                ref_ch_aligned = core.align_frames(ref_ch_data.frames,
                                             path=ref_ch_data.path,
                                             save_aligned=True, register=False,
                                             saved_shifts=spots_ch_data.shifts)
                print('Frames aligned')

        if verbose>0 and verbose<=3:
            print('\n')
            print(f'Data shape = {spots_ch_data.frames.shape}')
            print(f'SizeT = {spots_ch_data.SizeT}, '
                  f'SizeZ = {spots_ch_data.SizeZ}')
            print(f'Z resolution limit = {z_resolution_limit}')
            print(f'Gaussian filter sigma = {sigma}')
            print(f'Voxel dimensions = {vox_dim}')
            print(f'Numerical Aperture = {NA}')
            print(f'{channel_name} emission wavelength = {wavelen} nm')
            print(f'Filter good peaks method = {gop_how}')
            print(f'Filter good peaks limit = {gop_thresh_val[0]}')
            print('')

        if do_save:
            mitoQ_inputs_path = mitoQ_inputs_widget.save_analysis_inputs(
                                                    data_path, vNUM, df_inputs)
        


##################################################
Analysing experiment: G:/My Drive/1_MIA_Data/Dimitra/20200805_DCY008_8_YPD_M1

----------------------------------
Analysing experiment 1/1, Position_1 (1/12)
G:/My Drive/1_MIA_Data/Dimitra/20200805_DCY008_8_YPD_M1/TIFFs/Position_1...

Loading data...


Data shape = (25, 700, 700)
SizeT = 1, SizeZ = 25
Z resolution limit = 1.0
Gaussian filter sigma = 0.75
Voxel dimensions = [0.29, 0.07206, 0.07206]
Numerical Aperture = 1.4
Alexa Fluor 647 emission wavelength = 668.0 nm
Filter good peaks method = effect size
Filter good peaks limit = 0.2


----------------------------------
Analysing experiment 1/1, Position_2 (2/12)
G:/My Drive/1_MIA_Data/Dimitra/20200805_DCY008_8_YPD_M1/TIFFs/Position_2...

Loading data...


Data shape = (25, 700, 700)
SizeT = 1, SizeZ = 25
Z resolution limit = 1.0
Gaussian filter sigma = 0.75
Voxel dimensions = [0.29, 0.07206, 0.07206]
Numerical Aperture = 1.4
Alexa Fluor 647 emission wavelength = 668.0 nm
Filter goo